In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# Initialize model
model = TheModelClass()

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[0.6445, 0.7016, 1.0737,  ..., 1.0608, 1.0337, 0.4079],
           [0.2224, 0.7096, 0.4253,  ..., 0.7059, 1.0912, 1.1939],
           [0.3262, 0.5833, 0.1829,  ..., 0.5954, 1.0931, 0.9261],
           ...,
           [0.0000, 0.6969, 0.6909,  ..., 1.0786, 0.0437, 0.7785],
           [0.4589, 0.8038, 0.3040,  ..., 0.0000, 0.4004, 0.3798],
           [1.1150, 0.0000, 1.3144,  ..., 0.2682, 0.3149, 1.5613]],
 
          [[0.9268, 0.9333, 0.8470,  ..., 1.1632, 0.8317, 0.4549],
           [0.0747, 0.7007, 0.8653,  ..., 1.0888, 0.3255, 0.8984],
           [0.3957, 0.8363, 0.6087,  ..., 0.4042, 1.0745, 1.2923],
           ...,
           [2.8959, 0.3515, 0.0420,  ..., 0.6406, 0.5662, 0.1769],
           [0.0000, 0.7103, 0.5869,  ..., 0.0000, 0.7889, 0.5959],
           [1.1510, 0.5512, 1.3242,  ..., 0.5007, 0.7953, 0.0708]],
 
          [[1.1097, 0.7995, 1.0622,  ..., 0.0000, 0.4222, 0.4406],
           [1.0418, 0.1645, 1.2063,  ..., 0.9238, 0.6548, 0.3594],
           [1.0578, 1.00

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    # classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        # torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                a=0,
                                mode='fan_out',
                                nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


